# Trabalho Prático 1 - Aplicação de redes neuronais a um novo dataset
Objetivo: Explorar o uso de redes neuronais (keras) e comparação dos resultados com o uso de [AutoKeras](https://autokeras.com/).

Tipo de tarefa machine learning: Classificação Binária.

Dataset utilizado: [Titanic](https://public.opendatasoft.com/explore/dataset/titanic-passengers)

Repositório: [GitHub](https://github.com/spamz23/Applied-Deep-Learning/tree/005e14f8c27e176ac310183a5a2216c8838f2abc/Practical%20Work%201)

Trabalho realizado por:
* [Diogo Silva](https://github.com/spamz23)
* [Bruno Silva](https://github.com/brunosilva5)

# Instalar dependências

In [3]:
!pip install -r requirements.txt
# Limpar output
from IPython.display import clear_output
clear_output()

# Resultados Reproduzíveis
Para uma comparação justa entre os resultados de 2 técnicas diferentes, é importante que os resultados de ambas as técnicas sejam **reproduzíveis**. Para isto podemos fazer "seed" nos geradores de números aleatórios usados no backend da biblioteca `keras`.

In [2]:
global_seed = 0 # pode ser qualquer número
from numpy.random import seed
seed(global_seed)
import tensorflow as tf
tf.random.set_seed(global_seed)

ModuleNotFoundError: No module named 'tensorflow'

## 1. Fazer load dos dados
Para fazer load e tratar os dados, será utilizada a biblioteca pandas.

In [ ]:
import pandas as pd

index = "PassengerId"
dataset = pd.read_excel("titanic-passengers.xlsx", index_col="PassengerId")


### Primeiro é importante analisar e perceber a estrutura dos dados

In [ ]:
dataset

### Informações obtidas
Na tabela acima podemos observar que o dataset de treino é composto por 891 linhas (cada linha corresponde a uma pessoa), e por 11 colunas.
Podemos então tirar as seguintes conclusões:
1. Existem várias colunas com variáveis categóricas (`Name`, `Sex`, `Ticket`, `Cabin`, `Embarked`). Todas estas terão que ser convertidas para números para se poderem usar na rede neuronal.
2. Existem algumas colunas com valores `NaN`, ou seja valores em falta. Todas estas também terão que se preencher, ou então remover do dataset.
3. A coluna `Survived`, tem apenas valores binários (`0` e `1`). Esta indica se a pessoa em questão sobreviveu (`1`) ou não (`0`). O objetivo da rede neuronal será prever os valores desta coluna. Esta coluna terá que ser separada das restantes, para que seja utilizada como output.


# 2. Tratamento dos dados
Tal como referido anteriormente existem "problemas" nos dados, que terão de ser resolvidos antes de os aplicar numa rede neuronal.

## 2.1 Valores em falta

Para saber quais as colunas que contêm valores `NaN` podemos utilizar o seguinte código:

In [ ]:
print(f"Colunas com valores 'NaN': {dataset.columns[dataset.isna().any()].tolist()}")

Podemos observar que existem 3 colunas com valores `NaN`: `Age`, `Cabin` e `Embarked`

### 2.1.1 Coluna 'Cabin'

In [ ]:
missing = dataset["Cabin"].isnull().sum(axis=0)
total = dataset.shape[0]

print(f"Percentagem de valores em falta na coluna 'Cabin': {(missing*100)/total:.2f} %")

Como 77.10 % dos dados, na coluna `Cabin`, estão em falta, iremos proceder à remoção desta coluna, pois a sua inclusão provavelmente não se iria traduzir num ganho de informação na rede neuronal.

In [ ]:
dataset.drop(columns=['Cabin'], inplace=True)

### 2.1.2 Coluna 'Embarked'

In [ ]:
missing = dataset["Embarked"].isnull().sum(axis=0)
total = dataset.shape[0]

print(f"Percentagem de valores em falta na coluna 'Embarked': {(missing*100)/total:.2f} %")

Como nesta coluna apenas estão em falta apenas 0.22 % dos dados (o que corresponde a 2 linhas) e trata-se de uma variável categórica, iremos preencher os valores em falta com o valor mais frequente nesta coluna. Para isto iremos utilizar o módulo [SimpleImputer](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer) disponibilizado pela biblioteca [scikit-learn](https://scikit-learn.org/).

In [ ]:
from sklearn.impute import SimpleImputer
import numpy as np
# Criar objeto imputer com estratégia 'most_frequent'
imp_most_frequent = SimpleImputer(missing_values=np.nan, strategy='most_frequent', verbose=0)
# Aplicar na coluna 'Embarked'. Como vamos aplicar apenas a uma coluna, e o método espera um array 2D, é necessário fazer um reshape dos dados
dataset["Embarked"] = imp_most_frequent.fit_transform(dataset["Embarked"].values.reshape(-1, 1))

### 2.1.3 Coluna 'Age'

In [ ]:
missing = dataset["Age"].isnull().sum(axis=0)
total = dataset.shape[0]

print(f"Percentagem de valores em falta na coluna 'Age': {(missing*100)/total:.2f} %")

Nesta coluna estão em falta 19.87 % e trata-se de uma variável numérica, iremos preencher os valores em falta com o valor médio da coluna `Age`. Para isto iremos utilizar o mesmo módulo utilizado na coluna anterior. Existem 2 metodologias possíveis para o preenchimento de variáveis numéricas:
1. Utilizar a média dos valores da coluna;
2. Utilizar a mediana dos valores da coluna.
Para decidir qual a melhor estratégia é necessário observar a curva de distribuição dos dados desta coluna (`Age`).

In [ ]:
dataset["Age"].plot.kde()

Como é possível observar na figura acima, os dados não seguem uma distribuição normal. Assim concluímos que será mais vantajoso utilizar a mediana para o preenchimento dos valores em falta.

In [ ]:
from sklearn.impute import SimpleImputer
import numpy as np
# Criar objeto imputer com estratégia 'median'
imp_median = SimpleImputer(missing_values=np.nan, strategy='median', verbose=0)
# Aplicar na coluna 'Age'. Como vamos aplicar apenas a uma coluna, e o método espera um array 2D, é necessário fazer um reshape dos dados
dataset["Age"] = imp_median.fit_transform(dataset["Age"].values.reshape(-1, 1))

## 2.2 Transformar valores categóricos em valores numéricos

Para saber quais as colunas que contêm valores não numéricos podemos utilizar o seguinte código:

In [ ]:
non_numeric_columns = dataset.columns.difference(dataset._get_numeric_data().columns).to_list()
print(f"Colunas não númericas: {non_numeric_columns}")

### 2.2.1 Coluna 'Embarked', 'Sex', 'Name', 'Ticket'
Em primeiro lugar iremos remover as colunas `Name` e `Ticket`, pois estas são únicas dos passageiros.
Para transformar os valores das restantes colunas (`Embarked`, `Sex`) em valores numéricos iremos utilizar o método `get_dummies()` da biblioteca `pandas`. Este método irá criar, para cada coluna, novas `n` colunas, onde `n` é o numero de diferentes classes presentes em cada coluna. Estas novas colunas irão conter valores binários que indicam se uma linha pertence ou não a essa classe. 

Uma outra alternativa mais simples seria transformar, para cada coluna, cada classe diferente num valor inteiro diferente. Apesar de ser mais simples, esta tranformação poderia introduzir uma falsa sensação de hierarquiedade nos dados, o que poderia levar a rede neuronal a aprender falsas suposições. 

In [ ]:
dataset.drop(columns=["Name", "Ticket"], inplace=True)
# Fazer OneHotEncoding para todas as colunas não numéricas, excepto a coluna 'Survived'
dataset = pd.get_dummies(dataset, columns=["Embarked", "Sex"])

### 2.2.2 Coluna 'Survived'
As transformações a esta coluna são simples. Esta apenas contêm os valores `Yes` ou `No`. Apenas temos que transformar estes valores para `1` ou `0`, respetivamente.

In [ ]:
dataset["Survived"] = dataset['Survived'].map({'Yes': 1, 'No': 0})

## 2.3 Criar novos 'features' a partir de existentes
No dataset existe a coluna `SibSp` que indica o número de irmãos/cônjuges, do passageiro, a bordo. Existe também a coluna `Parch` que indica o número de pais/filhos, do passageiro, a bordo.
Como estas 2 colunas estão intrinsecamente relacionadas, decidimos uni-las numa só coluna, a que chamamos de `Family_Members`, que indica o número total de familiares a bordo do navio.

In [ ]:
# União das colunas
dataset["Family_Members"] = dataset["SibSp"] + dataset["Parch"]
# Apagar colunas que já não são necessárias
dataset.drop(columns=["SibSp", "Parch"], inplace=True)

## 2.4 Separação da coluna 'Survived'
É também importante separar a coluna `Survived` do resto do dataset. Iremos criar 2 variáveis,  `X`  (irá conter os dados), e `y` (irá conter a coluna `Survived`)

In [ ]:
X = dataset[dataset.columns.difference(["Survived"])]
y = dataset[["Survived"]]

## 2.5 Normalização de dados
Para que o modelo tenha uma maior performance, por vezes, é fundamental fazer a normalização dos dados. 
O objetivo da normalização é alterar os valores das colunas numéricas no conjunto de dados para uma escala comum, sem distorcer as diferenças no intervalos de valores.
Neste dataset, como existe uma diferença significativa na escala dos valores, como por exemplo, na coluna `Pclass`, onde os valores variam entre 1 e 3, e as colunas `Age` e `Fare` que variam de `0` a `80` e `0` a `512`, respetivamente, é fundamental normalizar os dados pois os intervalos são muito diferentes.

In [ ]:
from sklearn.preprocessing import StandardScaler

# Definir colunas a normalizar
cols = ["Pclass", "Age", "Family_Members", "Fare"]
# Normalizar os dados
scaled_columns = StandardScaler().fit_transform(X[cols].values)
# Juntar os dados normalizados com os outros
X = np.concatenate([X[X.columns.difference(cols)], scaled_columns], axis=1)


## 2.6 Divisão dos dados

É fundamental fazer a divisão do dataset em dados de treino e dados de teste, de modo a poder avaliar a verdadeira capacidade de generalização, esta deve conseguir prever corretamente dados que nunca tenha visto.
Uma metodologia bastante comum, é usar 80% dos dados para treinar o modelo e os restantes para avaliar a sua qualidade.

In [ ]:
from sklearn.model_selection import train_test_split
# Dividir dados em treino e teste. 
# 20% dos dados são usados para teste
# random_state = 42, para resultados reproduzíveis
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = global_seed, stratify=y)

# 3. Construir e treinar as Redes Neuronais

In [ ]:
# Método para apresentar diferentes métricas de avaliação dos modelos
def print_test_results(y_test, y_predict_proba):
    # Gerar previsões de classes
    y_predict = (y_predict_proba > 0.5).astype(np.int32)
    # Comparar previsões com valores reais
    print("Métricas obtidas nos dados de teste:")
    print(f"ROC Auc: {sk_metrics.roc_auc_score(y_test, y_predict_proba):.2f}")
    print(f"Accuracy: {sk_metrics.accuracy_score(y_test, y_predict):.2f}")
    print(f"F1 Score: {sk_metrics.f1_score(y_test, y_predict):.2f}")
    print(f"Precision: {sk_metrics.precision_score(y_test, y_predict):.2f}")

## 3.1 Construção manual de rede neuronal Keras

In [ ]:
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.layers import Dense
from keras import metrics
from sklearn import metrics as sk_metrics


# Criar modelo sequencial
classic_model = Sequential()
# Criar camadas 'Dense'
classic_model.add(Dense(8, input_dim = X_train.shape[1], activation = 'relu'))
classic_model.add(Dense(8, activation = 'relu'))
# Ativação = 'sigmoid' para os valores de saída serem 0 ou 1
classic_model.add(Dense(1, activation = 'sigmoid'))
# Compilar modelo
# Optimizar métrica 'AUC' para melhores resultados em classificação binária 
classic_model.compile(loss = 'binary_crossentropy', optimizer = 'adam')
# Treinar modelo
classic_model.fit(X_train, y_train, epochs = 120, verbose=0)
# Fazer previsões de probabilidades
y_predict_proba = classic_model.predict(X_test)
# Apresentar resultados
print_test_results(y_test, y_predict_proba)



## 3.2 Uso do `AutoKeras`
A biblioteca `AutoKeras` é uma biblioteca de AutoML (*Automated Machine Learning*), e pretende tornar o uso de *machine learning* acessível a todos. 

O seu objetivo é abstrair um utilizador da escolha dos vários hyper-parâmetros de uma rede neuronal, bem como da sua arquitetura. Esta biblioteca experimenta `n` redes com configurações diferentes, e escolhe automaticamente a que melhor se adapta aos dados, ou seja a que tiver melhor capacidade de generalização.

In [ ]:
import autokeras as ak

# Construir modelo
auto_model = ak.StructuredDataClassifier(
    overwrite=True,
    seed=global_seed,
    max_trials=20) # Experimenta 20 modelos diferentes

# Prepapar dados
X_train = X_train.astype('float32')
y_train = y_train.astype('float32')
X_test = X_test.astype('float32')
y_test = y_test.astype('float32')
# Treinar modelo
auto_model.fit(X_train, y_train, epochs=120)
# Exportar modelo AutoKeras para modelo Keras
auto_model = auto_model.export_model()

# Prever probabilidades
y_predict_proba = auto_model.predict(X_test)
# Apresentar resultados
print_test_results(y_test, y_predict_proba)



## 3.3 Comparação de resultados
Na seguinte tabela podemos observar as diferentes métricas obtidas por ambos os modelos:

| Modelo | ROC Auc | Accuracy | F1 Score | Precision
|:---:|:---:|:---:|:---:|:---:|
| Modelo `Keras` manual | 0.86 | 0.82 | **0.74** | 0.81 | 
| Modelo `AutoKeras` |  0.86 | 0.82 | 0.73 | **0.83** | 

Como é possível observar, ambos os modelos apresentaram resultados bastantes semelhantes, com diferenças minímas apenas no **F1 Score** e na **Precisão**.

No também pode ser também interessante observar ambas as arquiteturas das duas redes criadas.



**Estrutura Primeiro modelo:**

In [ ]:
# Resumir primeiro modelo
classic_model.summary()

**Estrutura Modelo `AutoKeras`:**

In [ ]:
# Resumir modelo `AutoKeras`
auto_model.summary()

**Diferenças encontradas:**

Como podemos observar a rede neuronal construída pela biblioteca `AutoKeras` apresenta um nível de complexidade muito superior à rede neuronal construída manualmente. Enquanto a primeira rede apresenta 161 parâmetros treináveis, a segunda conta com quase nove vezes mais parâmetros treináveis (1409).

Portanto no presente contexto não se justificaria a escolha da segunda rede neuronal, pois em regra geral, quando 2 ou mais modelos apresentam resultados bastante idênticos, deve-se adoptar o modelo mais simples. Isto leva a que:
* O modelo seja mais facilmente explicável;
* Menor tempo de treino, que por sua vez leva a menores custos de computação.

A adoção de uma rede neuronal construída manualmente oferece ainda a vantagem de um maior controlo sobre todo o processo. No entanto é de notar que foram experimentados apenas `20` modelos diferentes pelo `AutoKeras`. Um aumento no número de experiências realizadas poderia traduzir-se na construção de um melhor modelo. O uso do `AutoKeras` é também uma ferramenta aconselhável a iniciantes de *machine learning*, pois abstrai o utilizador dos complicados conceitos matemáticos intrínsecos deste tópico.
